Regular expressions are very useful in many situations, and not exclusive to Python.  In fact, once you grasp the concepts, you'll find them indispensible and use them (or miss) them for many programming and data management tasks.  This notebook intends to give you a flavor of the possibilities, it doesn't intend to be a comprehensive overview.

# Requirements

In Python, regular expressions are implemented in the standard library's `re` module.

In [4]:
import re

# Match making

One of the tasks regular expressions are useful for is verifying whether a (collection of) string(s) matches a certain pattern.

Example: for a list of file names, select only the ones that start with `dev_`, and end with `.txt`.

In [ ]:
file_list = ['dev_counter.txt', 'dev_reset.txt', 'shm_counter.txt', 'dev_start.txt']
for file_name in file_list:
    if re.match(r'dev_.*\.txt', file_name):
        print(file_name)

A somewhat more complex example, select file names that have a base name ending in digits, and extension either `.txt`, or `.dat`.

In [ ]:
file_list = ['exp_01.txt', 'exp.txt', 'exp_02.dat', 'exp.dat', 'exp05.dat', 'exp_03.jpg']
for file_name in file_list:
    if re.search(r'\d+\.(?:txt|dat)', file_name):
        print(file_name)

Is this really correct?  Let's try something nasty.

In [ ]:
if re.search(r'\d+\.(?:txt|dat)', 'exp_09.data'):
    print('Oops!')
else:
    print("Yay!")

Let's ensure that the strings have to end with either `.txt`, or `.dat`.

In [ ]:
if re.search(r'\d+\.(?:txt|dat)$', 'exp_09.data'):
    print('Oops!')
else:
    print("Yay!")

# Extracting stuff

Regular expressions can also be used to capture parts of a string while matching.

Suppose we are only interested in the numbers in file names like `exp_01.dat`.

In [ ]:
file_list = ['exp_01.dat', 'meta.txt', 'exp_02.dat', 'exp_10.dat', 'exp_05.dat', 'exp_03.jpg']
for file_name in file_list:
    match = re.search(r'exp_(\d+)\.dat', file_name)
    if match:
        print(match.group(1))

Note the difference between grouping brackets such as `(?:txt|dat)`, and capturing brackets such as `(\d+)`.  Capturing brackets also group, but grouping brackets don't capture.

# Substitution

Regular expressions can also be used to substitute parts of strings that match a given pattern.  For instance, replace all extensions in file names by `.txt`.

In [ ]:
file_list = ['exp_01.dat', 'exp_03.txt', 'exp_02.dat', 'exp_10.text']
for file_name in file_list:
    new_file_name = re.sub(r'\.\w+$', '.txt', file_name)
    print('{old:15s} -> {new}'.format(old=file_name, new=new_file_name))

The substituion can infact include part of the string captured in the regular expression.  We can replace a file name such as `exp_03.txt` by `03_exp.txt`, and `dev_05.dat` by `05_dev.dat`.

In [ ]:
file_list = ['exp_01.dat', 'dev_03.txt', 'exp_02.txt', 'exp_10.text']
for file_name in file_list:
    new_file_name = re.sub(r'(\w+)_(\d+)\.', r'\2_\1.', file_name)
    print('{old:15s} -> {new}'.format(old=file_name, new=new_file_name))

# Composition

Sophisticated regular expressions tend to be very hard to read.  There are a couple of things you can do to mitigate that issue.
* Use `re.VERBOSE` so that you can add whitespace and comments to the regular expression defintions.
* Use composition, i.e., define regular expressions that describe part of the match, and compose those t match the entire expression.

Consider the following example, a log message.  We want to extract the date-time information, the log level, the process number and the exit value.

In [26]:
log_entry = '2021-08-25 17:04:23.439405 [info]: end process 1 exited with 2'

Rather than writing a regular expression that describes the entire log message, we write expressions that match part of it.

In [33]:
date = r'\d{4}-\d{2}-\d{2}'
time = r'\d{2}:\d{2}:\d{2}\.\d+'

Let's check that the time matches.

In [34]:
match = re.search(time, log_entry)
match.group(0)

'17:04:23.439405'

We cna now use `date` and `time` to match the entire date-time value.

In [45]:
regex = re.compile(r'({date}\s+{time})'.format(date=date, time=time))
match = regex.search(log_entry)
match.group(1)

'2021-08-25 17:04:23.439405'

In [46]:
level = r'\[(\w+)\]'
msg = r'end\s+process\s+(\d+)\s+exited\s+with\s+(\d+)'

In [47]:
regex = re.compile(r'({date}\s+{time})\s+{level}\s*:\s*{msg}'.format(date=date, time=time, level=level, msg=msg))
match = regex.match(log_entry)
print(f'datetime = {match.group(1)}')
print(f'log level: {match.group(2)}')
print(f'process = {match.group(3)}')
print(f'exit status = {match.group(4)}')

datetime = 2021-08-25 17:04:23.439405
log level: info
process = 1
exit status = 2


Although the final regular expression is still rather long, it is easier to read and to maintain.  Using `re.VERBOSE` and triple-quoted strings helps to further make the regular expression more maintainable.

In [48]:
regex = re.compile(r'''
    ({date}\s+{time})\s+        # date-time, up to microsecond presision
    {level}\s*:\s*              # log level of the log message
    {msg}                       # actual log message
    '''.format(date=date, time=time, level=level, msg=msg), re.VERBOSE)
match = regex.match(log_entry)
print(f'datetime = {match.group(1)}')
print(f'log level: {match.group(2)}')
print(f'process = {match.group(3)}')
print(f'exit status = {match.group(4)}')

datetime = 2021-08-25 17:04:23.439405
log level: info
process = 1
exit status = 2
